In [ ]:
import sys
import pandas as pd
from matplotlib import pyplot as plt
from dataclasses import dataclass
import numpy as np
from typing import Optional, List
from backtest_broker import Position, Broker
from copy import deepcopy
from backtest import BackTestResults
from loguru import logger

logger.remove()
logger.add(sys.stderr, level="ERROR")

%load_ext autoreload
%autoreload 2

In [ ]:
@dataclass
class FeeConfig:
    broker: Optional[float] = None
    slipage: Optional[float] = None
    W: Optional[float] = None
    R: Optional[float] = None
    base: Optional[float] = None

class Comission():
    def __init__(self, config:FeeConfig):
        self.params = config

    def _calc_stock_comission(self, price):
        return round(price*self.params.W/self.params.R*self.params.base/100, 2)

    def __call__(self, price, is_taker:bool):
        com = self.params.broker
        if is_taker:
            com += self._calc_stock_comission(price) + self.params.slipage
        return com
    
fee_calculator = Comission(FeeConfig(broker=0.7, slipage=1, W=1, R=1, base=0.00462))

In [ ]:
df = pd.read_csv("/Users/andrybin/Yandex.Disk.localized/Data/validation/DMA@SR-M15.txt")
# df["Volume"] = df.Type.abs()
# df.Type /= df.Volume
df.Time = list(map(np.datetime64, pd.to_datetime(df.Time)))
df

In [ ]:
# profits, fees = [0], [0]
last_type = 0
last_pos = None
#last_pos = Position(df.Price.iloc[0], df.Time.iloc[0],0,
#              #             volume=100000/df.Price.iloc[0]*6, 
#                #            fee_rate=0.003                                    )
positions = []

for i in range(0, len(df)):
    # cur_prof = last_type*(df.Price.iloc[i]-df.Price.iloc[i-1])
    # cur_fee = fee_calculator(df.Price.iloc[i-1], df.SL.iloc[i] > 0)
    # print(last_type, cur_prof, df.SL.iloc[i], cur_fee)
    last_type = df.Type.iloc[i] + last_type
    if last_pos is not None:
        last_pos.close(df.Price.iloc[i], df.Time.iloc[i], i)
    # profits.append(cur_prof)
    # fees.append(cur_fee)
    if last_pos is not None:
        positions.append(last_pos)
    last_pos = None
    if last_type != 0:
        last_pos = Position(df.Price.iloc[i]*np.sign(last_type), np.datetime64(df.Time.iloc[i]), i, volume=100000/df.Price.iloc[i], fee_rate=0.003)

In [ ]:
br = Broker(None)
br.positions = positions

bres = BackTestResults("2015-01-01", "2024-06-01", 100000)
bres.wallet = 100000
bres.leverage = 6
bres.process_backtest(br)

In [ ]:
bres.APR, bres.final_profit, bres.metrics

In [ ]:
    fig, ax1 = plt.subplots(figsize=(15, 8))
    ax2 = ax1.twinx()
    ax1.plot(bres.daily_hist.days, bres.daily_hist.profit, linewidth=3, color="b", alpha=0.6)
    ax1.plot(bres.deal_hist.dates, bres.deal_hist.profit, linewidth=1, color="b", alpha=0.6)    
    ax1.plot(bres.deal_hist.dates, bres.deal_hist.profit_nofees, linewidth=1, color="r", alpha=0.6)
    if "buy_and_hold" in bres.daily_hist.columns:
        ax1.plot(bres.daily_hist.days, bres.daily_hist.buy_and_hold, linewidth=1, alpha=0.6)  
    ax1.legend(["sum. profit", "profit from strategy", "profit without fees", "buy and hold"])
    plt.grid("on")
    ax2.plot(bres.daily_hist["days"], bres.daily_hist["deposit"], "-", linewidth=3, alpha=0.3)
    ax2.legend(["deposit"])
    
    plt.tight_layout()
    plt.savefig("backtest.png")
    plt.show()

In [ ]:
daily_hist_lkoh = bres.daily_hist

In [ ]:
daily_hist_sr = bres.daily_hist

In [ ]:
plt.plot(daily_hist_lkoh.days, daily_hist_lkoh.profit, linewidth=3, color="b", alpha=0.6)
plt.plot(daily_hist_sr.days, daily_hist_sr.profit, linewidth=3, color="g", alpha=0.6)
profit_av = (daily_hist_sr.profit+daily_hist_lkoh.profit)/2
plt.plot(daily_hist_sr.days, profit_av, linewidth=1, color="r")

In [ ]:
APR_av, maxwait_av = bres.metrics_from_profit(profit_av.values)
APR_av, maxwait_av